In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic
from surprise.model_selection import cross_validate

import pprint

In [2]:
ratings_df = pd.read_csv("rating.csv")
ratings_df.shape

(7813737, 3)

In [3]:
ratings_df.head(5)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [4]:
# ratings_df["rating"].unique()

total_ratings_count = ratings_df["rating"].value_counts()
print(total_ratings_count)

rating
 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: count, dtype: int64


In [5]:
# ratings_df = ratings_df[ratings_df.rating!=-1]

In [6]:
# ratings_df.shape

In [7]:
anime_df = pd.read_csv("anime.csv")
anime_df.shape

(12294, 7)

In [8]:
anime_df.head(5)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [9]:
df = ratings_df.merge(anime_df.drop_duplicates(), left_on="anime_id", right_on= "anime_id", how="inner")
df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395
...,...,...,...,...,...,...,...,...,...
7813722,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266
7813723,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747
7813724,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895
7813725,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005


In [10]:
df.shape

(7813727, 9)

In [11]:
df =df.dropna()
df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395
...,...,...,...,...,...,...,...,...,...
7813722,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266
7813723,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747
7813724,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895
7813725,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005


In [12]:
df.shape

(7813611, 9)

In [13]:
df = df[df.rating_x!=-1]

In [14]:
df.shape

(6337146, 9)

In [15]:
total_ratings_count = df["rating_x"].value_counts()
print(total_ratings_count)

rating_x
8     1646018
7     1375277
9     1254090
10     955712
6      637762
5      282791
4      104275
3       41437
2       23140
1       16644
Name: count, dtype: int64


In [16]:
df["rating_x"].describe()

count    6.337146e+06
mean     7.808543e+00
std      1.572440e+00
min      1.000000e+00
25%      7.000000e+00
50%      8.000000e+00
75%      9.000000e+00
max      1.000000e+01
Name: rating_x, dtype: float64

## Samplen ottaminen

In [17]:
# df = df.sample(frac=0.0001, random_state=42) 
df = df.sample(n=1000, random_state=42).reset_index(drop=True)  # Sample 6000 rows
# df = df.sample(n=100000, random_state=42).reset_index(drop=True)  # Sample 6000 rows
# df = df.head(100000)
# # len(df)

In [18]:
print(df.head())  # Check the sampled data
print(df.index)   # Ensure the index is reset (should start from 0)

   user_id  anime_id  rating_x                           name  \
0    27383      1557         6               Shounen Onmyouji   
1     5899     12365         6            Bakuman. 3rd Season   
2    31017       157         7           Mahou Sensei Negima!   
3    53005       199         8  Sen to Chihiro no Kamikakushi   
4    25989     10721         9             Mawaru Penguindrum   

                                               genre   type episodes  \
0  Action, Demons, Fantasy, Historical, Magic, Sh...     TV       26   
1                    Comedy, Drama, Romance, Shounen     TV       25   
2  Comedy, Ecchi, Fantasy, Harem, Magic, Romance,...     TV       26   
3                     Adventure, Drama, Supernatural  Movie        1   
4              Comedy, Drama, Mystery, Psychological     TV       24   

   rating_y  members  
0      7.71    33321  
1      8.71   133620  
2      7.11    76253  
3      8.93   466254  
4      8.08   156689  
RangeIndex(start=0, stop=1000, step=1)

In [19]:
df.columns

Index(['user_id', 'anime_id', 'rating_x', 'name', 'genre', 'type', 'episodes',
       'rating_y', 'members'],
      dtype='object')

In [20]:
df.shape

(1000, 9)

In [21]:
mean_rating = df.groupby("anime_id") [["rating_x"]].mean()
mean_rating.sort_values(by="rating_x", ascending=False)

,rating_x
anime_id,
11843,10.0
834,10.0
15583,10.0
873,10.0
16051,10.0
...,...
916,3.0
2890,3.0
5530,3.0


In [22]:
df["rating_x"].describe()

count    1000.000000
mean        7.786000
std         1.592086
min         1.000000
25%         7.000000
50%         8.000000
75%         9.000000
max        10.000000
Name: rating_x, dtype: float64

## Mallinnus

In [23]:
lukija = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(df[["user_id", "name", "rating_x"]], lukija)
data

In [24]:
# Set all data as training set >> tämän voi käyttää, jos riittävästi dataa
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25)
print(trainset)
print(testset)

[(33650, 'Murder Princess', 8.0), (29135, 'Clannad', 10.0), (4004, 'Yu☆Gi☆Oh!: Duel Monsters GX', 8.0), (67829, 'Re:Zero kara Hajimeru Isekai Seikatsu', 9.0), (58587, 'Gantz 2nd Stage', 8.0), (46373, 'Tamako Love Story', 8.0), (55313, 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen', 8.0), (41084, 'Sonic X', 6.0), (8346, 'Cowboy Bebop: Tengoku no Tobira', 8.0), (44887, 'FLCL', 8.0), (43701, 'Naruto: Shippuuden Movie 5 - Blood Prison', 8.0), (72814, 'Steins;Gate Movie: Fuka Ryouiki no Déjà vu', 10.0), (47531, 'Clannad', 9.0), (49173, 'Ghost in the Shell: Stand Alone Complex - Tachikoma na Hibi', 7.0), (58903, 'Elfen Lied Special', 10.0), (49408, 'Star Ocean EX', 6.0), (48302, 'Kimi no Iru Machi', 7.0), (47175, 'School Rumble', 7.0), (67749, 'Kaichuu!', 6.0), (11858, 'Hajime no Ippo', 10.0), (27671, 'Detective Conan Movie 06: The Phantom of Baker Street', 8.0), (70854, 'Prison School', 8.0), (18985, 'Hikaru no Go', 8.0), (67086, 'Strike the Blood', 9.0), (53337, 'Slayers: The Moti

In [25]:
# Build and train an algorithm.
# # Tarjotaan trainset algoritmille

# sim_options = {
#     "name": "cosine",
#     "user_based": True,
#     "min_support": True,
#     "min_K": 3
#     }


sim_options = {"user_based": True} # False compute similarities between items, True between customers

algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [26]:
preds = algo.test(testset)
preds[0:20]

[Prediction(uid=33650, iid='Murder Princess', r_ui=8.0, est=7.781333333333333, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=29135, iid='Clannad', r_ui=10.0, est=7.781333333333333, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=4004, iid='Yu☆Gi☆Oh!: Duel Monsters GX', r_ui=8.0, est=7.781333333333333, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=67829, iid='Re:Zero kara Hajimeru Isekai Seikatsu', r_ui=9.0, est=7.781333333333333, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=58587, iid='Gantz 2nd Stage', r_ui=8.0, est=7.781333333333333, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=46373, iid='Tamako Love Story', r_ui=8.0, est=7.781333333333333, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=55313, iid='Rurouni 

In [27]:
user_id_to_search = 4341  # Replace with the specific user_id
user_data = df[df['user_id'] == user_id_to_search]

print(user_data)


Empty DataFrame
Columns: [user_id, anime_id, rating_x, name, genre, type, episodes, rating_y, members]
Index: []


Mean Absolute Error (MAE)
Root Mean Squared Error (RMSE):
The algorithm shows moderate variability across the three folds, with an average RMSE of approximately 1.6, indicating the typical magnitude of error. MAE provides a complementary perspective, indicating an average absolute error of approximately 1.26.

In [28]:
cross_validate(algo, data, measures=["MAE", "RMSE"], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE, RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     1.3338  1.1914  1.2396  1.2549  0.0591  
RMSE (testset)    1.6972  1.4947  1.5774  1.5898  0.0831  
Fit time          0.01    0.01    0.00    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    


{'test_mae': array([1.3337649 , 1.19144032, 1.23963244]),
 'test_rmse': array([1.69719011, 1.49469572, 1.57740926]),
 'fit_time': (0.007548332214355469,
  0.006983280181884766,
  0.004953861236572266),
 'test_time': (0.002207040786743164,
  0.0010006427764892578,
  0.0010051727294921875)}

In [29]:
from collections import Counter

# Analyze preds for impossible predictions
was_impossible = [pred.details.get('reason', 'Unknown') for pred in preds if pred.details.get('was_impossible')]

# Count the reasons
reason_counts = Counter(was_impossible)

# Output results
print("Total 'was_impossible=True':", len(was_impossible))
print("Reasons and their counts:")
for reason, count in reason_counts.items():
    print(f"{reason}: {count}")



Total 'was_impossible=True': 250
Reasons and their counts:
User and/or item is unknown.: 248
Not enough neighbors.: 2


In [30]:
# Filter out predictions where 'was_impossible' is True
filtered_preds = [pred for pred in preds if not pred.details.get('was_impossible', False)]

# Check the number of filtered predictions
print("Total predictions after filtering:", len(filtered_preds))


Total predictions after filtering: 0


### SVD yritys

In [31]:
# from surprise import SVD
# import sys
# import surprise

# from recommenders.utils.timer import Timer
# from recommenders.datasets import movielens
# from recommenders.datasets.python_splitters import python_random_split
# from recommenders.evaluation.python_evaluation import (
#     rmse,
#     mae,
#     rsquared,
#     exp_var,
#     map_at_k,
#     ndcg_at_k,
#     precision_at_k,
#     recall_at_k,
#     get_top_k_items,
# )
# from recommenders.models.surprise.surprise_utils import (
#     predict,
#     compute_ranking_predictions,
# )
# from recommenders.utils.notebook_utils import store_metadata


# print(f"System version: {sys.version}")
# print(f"Surprise version: {surprise.__version__}")



In [32]:
# train, test = python_random_split(df, 0.75)